In [19]:
import pandas as pd
import xml.etree.ElementTree as ET
import os

from helper_functions import *

In [20]:
# Loading the lookup table
df_lookup = pd.read_csv('../input/store_to_pool_lookup.csv', dtype=str)
df_lookup.head()

,Store#,Pool_ID,Pool#,AddressName
0,0001,FRA,2693,Bed Bath and Beyond #2693\nc/o Ryder Integrate...
1,0006,SPR,2690,Palmer Dedicated Logistics
2,0008,SCA,2613,Bed Bath and Beyond #2613 c/o Taylored Services
3,0009,FRA,2693,Bed Bath and Beyond #2693\nc/o Ryder Integrate...
4,0012,SPR,2690,Palmer Dedicated Logistics


In [21]:
curr_dir = os.getcwd()
print(curr_dir)
root_dir = os.path.dirname(os.getcwd())
print(root_dir)

g:\My Drive\Cirkul_Repository\Data_and_Analytics\GitHub_Cirkul\ASN_Pool_Fix\src
g:\My Drive\Cirkul_Repository\Data_and_Analytics\GitHub_Cirkul\ASN_Pool_Fix


In [22]:
# Getting the current proejct directory
project_dir = os.path.dirname(os.getcwd())
project_dir = project_dir.replace('\\','/')
selected_folder = 'sample'

files = [file for file in os.listdir('../input/'+ selected_folder + '/')]
# inputFolderPath = 'G:/My Drive/Cirkul_Repository/Data_and_Analytics/GitHub_Cirkul/ASN_Pool_Fix/input/' + selected_folder + '/'
inputFolderPath = project_dir + "/input/" + selected_folder + "/"

for file in progressbar(files, "Progress: ", 60):
    inputFilePath = inputFolderPath + file
    tree = ET.parse(inputFilePath)
    root = tree.getroot()

    # Finding the reference ID
    shipment = root.find('{http://www.spscommerce.com/RSX}Shipment')
    header = shipment.find('{http://www.spscommerce.com/RSX}Header')
    address = header.find('{http://www.spscommerce.com/RSX}Address')
    addrLocation = address.find('{http://www.spscommerce.com/RSX}AddressLocationNumber')
    addrName = address.find('{http://www.spscommerce.com/RSX}AddressName')

    # Finding the Address Loaction  Number from the lookup table
    matchFound = "No"
    for i in df_lookup.index:
        if str(df_lookup.loc[i,"Store#"]) == addrLocation.text:
            matchFound = "Yes"
            newAddrLocation = str(df_lookup.loc[i,"Pool#"])
            newAddrName = str(df_lookup.loc[i,"AddressName"])
    
    if matchFound == "No":
        print(addrLocation.text + " : Match not found")
        print(addrLocation.text)

    # Updating the Address Location and Name
    addrLocation.text = newAddrLocation
    addrName.text = newAddrName

    # Exporting the modified file
    outputFolderPath = project_dir + "/output/"
    ouputFilePath = outputFolderPath + file
    tree.write(ouputFilePath, 'utf-8', True, 'http://www.spscommerce.com/RSX')

g:/My Drive/Cirkul_Repository/Data_and_Analytics/GitHub_Cirkul/ASN_Pool_Fix
g:/My Drive/Cirkul_Repository/Data_and_Analytics/GitHub_Cirkul/ASN_Pool_Fix/input/sample/
Progress: [############################################################] 1/1

